写一个小程序，3，5，15，fizz，buzz，fizzbuzz


In [4]:
def fizz_buzz_encode(i):
    if i%15 == 0:return 3
    elif i%5 == 0:return 2
    elif i%3 == 0:return 1
    else:
        return 0

def fizz_buzz_decode(i,prediction):
    return [str(i),"fizz",'buzz','fizzbuzz'][prediction]

def helper(i):
    print(fizz_buzz_decode(i,fizz_buzz_encode(i)))
    
for i in range(1,16):
    helper(i)

1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz


现在使用神经网络来学习这个过程


我们首先定义数据的输入输出

In [14]:
import numpy as np
import torch
NUM_DIGITS = 10
# 转2进制
def binary_encode(i,num_digits):
    return np.array([i >> d&1 for d in range(num_digits)][::-1])

trX = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(101,2**NUM_DIGITS)])
trY = torch.Tensor([fizz_buzz_encode(i) for i in range(101,2**NUM_DIGITS)])
print(binary_encode(30,NUM_DIGITS))

[0 0 0 0 0 1 1 1 1 0]


In [15]:
trY

tensor([0., 1., 0., 0., 3., 0., 0., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 0.,
        0., 3., 0., 0., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 0., 0., 3., 0.,
        0., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 0., 0., 3., 0., 0., 1., 0.,
        2., 1., 0., 0., 1., 2., 0., 1., 0., 0., 3., 0., 0., 1., 0., 2., 1., 0.,
        0., 1., 2., 0., 1., 0., 0., 3., 0., 0., 1., 0., 2., 1., 0., 0., 1., 2.,
        0., 1., 0., 0., 3., 0., 0., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 0.,
        0., 3., 0., 0., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 0., 0., 3., 0.,
        0., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 0., 0., 3., 0., 0., 1., 0.,
        2., 1., 0., 0., 1., 2., 0., 1., 0., 0., 3., 0., 0., 1., 0., 2., 1., 0.,
        0., 1., 2., 0., 1., 0., 0., 3., 0., 0., 1., 0., 2., 1., 0., 0., 1., 2.,
        0., 1., 0., 0., 3., 0., 0., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 0.,
        0., 3., 0., 0., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 0., 0., 3., 0.,
        0., 1., 0., 2., 1., 0., 0., 1., 

# 定义模型
本质上是一个分类问题，所以使用交叉熵函数

In [50]:
NUM_HIDDEN = 10
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS,NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN,4)
)
# if torch.cuda.is_available():
#     model = model.cuda()

In [79]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.1)#需要传参
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

模型训练代码

In [80]:
BATCH_SIZE = 128
for epoch in range(10000):
    for start in range(0,len(trX),BATCH_SIZE):
        
        end = start+BATCH_SIZE if (start+BATCH_SIZE) <= 10000 else 10000
        batchX = trX[start:end]
        batchY = trY[start:end]
        
#         if torch.cuda.is_available():
#             batchX = batchX.cuda()
#             batchY = batchY.cuda()
        y_pred = model(batchX)#forward
        loss = loss_fn(y_pred,batchY.long())
        if epoch %100 == 0:
            print("Epoch{}loss:{}".format(epoch,loss.item()))
        optimizer.zero_grad()
        
        loss.backward()#backward
        optimizer.step()#gradient descent

Epoch0loss:0.5737388134002686
Epoch0loss:0.8785625100135803
Epoch0loss:1.5970242023468018
Epoch0loss:1.4820877313613892
Epoch0loss:2.597548007965088
Epoch0loss:1.1283106803894043
Epoch0loss:1.7006829977035522
Epoch0loss:4.997381687164307
Epoch100loss:0.9698892831802368
Epoch100loss:1.1650962829589844
Epoch100loss:0.7178366780281067
Epoch100loss:0.8149125576019287
Epoch100loss:0.6310733556747437
Epoch100loss:0.724969208240509
Epoch100loss:0.9567988514900208
Epoch100loss:0.6940712928771973
Epoch200loss:1.4748226404190063
Epoch200loss:1.2774685621261597
Epoch200loss:0.715100884437561
Epoch200loss:0.6729467511177063
Epoch200loss:0.589329719543457
Epoch200loss:0.9040784239768982
Epoch200loss:1.8044500350952148
Epoch200loss:0.9966558814048767
Epoch300loss:1.705514907836914
Epoch300loss:1.3455328941345215
Epoch300loss:0.7511727213859558
Epoch300loss:0.6121840476989746
Epoch300loss:0.536142110824585
Epoch300loss:0.6366485357284546
Epoch300loss:0.7488676905632019
Epoch300loss:0.518187940120697


Epoch3100loss:0.5975028276443481
Epoch3100loss:0.5969968438148499
Epoch3100loss:0.7970011830329895
Epoch3100loss:0.46491169929504395
Epoch3200loss:1.2737581729888916
Epoch3200loss:1.0058170557022095
Epoch3200loss:0.7213157415390015
Epoch3200loss:0.6293328404426575
Epoch3200loss:0.5321873426437378
Epoch3200loss:0.7375555038452148
Epoch3200loss:1.426435947418213
Epoch3200loss:0.9364455342292786
Epoch3300loss:0.6112973690032959
Epoch3300loss:0.6704314351081848
Epoch3300loss:0.6113384366035461
Epoch3300loss:0.6821280121803284
Epoch3300loss:0.5371108055114746
Epoch3300loss:0.6300662755966187
Epoch3300loss:0.71455979347229
Epoch3300loss:0.43988820910453796
Epoch3400loss:1.249208927154541
Epoch3400loss:1.0151727199554443
Epoch3400loss:0.7194303274154663
Epoch3400loss:0.6324552297592163
Epoch3400loss:0.5331442952156067
Epoch3400loss:0.7313092947006226
Epoch3400loss:1.416279911994934
Epoch3400loss:0.9416314363479614
Epoch3500loss:0.7278602123260498
Epoch3500loss:0.8664301633834839
Epoch3500loss

Epoch6200loss:0.5918076634407043
Epoch6200loss:0.6252480745315552
Epoch6200loss:1.035539984703064
Epoch6200loss:0.7721683382987976
Epoch6300loss:1.0797206163406372
Epoch6300loss:0.8989899158477783
Epoch6300loss:0.6486903429031372
Epoch6300loss:0.5564299821853638
Epoch6300loss:0.4769682288169861
Epoch6300loss:0.6027399897575378
Epoch6300loss:1.0763623714447021
Epoch6300loss:0.7233020663261414
Epoch6400loss:0.5600733160972595
Epoch6400loss:0.5629279613494873
Epoch6400loss:0.5677783489227295
Epoch6400loss:0.6136394143104553
Epoch6400loss:0.5447880625724792
Epoch6400loss:0.597202718257904
Epoch6400loss:0.819367527961731
Epoch6400loss:0.7191399931907654
Epoch6500loss:1.1356128454208374
Epoch6500loss:1.102683186531067
Epoch6500loss:0.8624913692474365
Epoch6500loss:0.5822391510009766
Epoch6500loss:0.5136546492576599
Epoch6500loss:0.6320969462394714
Epoch6500loss:1.0607224702835083
Epoch6500loss:1.6933417320251465
Epoch6600loss:0.547981321811676
Epoch6600loss:0.5552996397018433
Epoch6600loss:0

Epoch9300loss:0.45384711027145386
Epoch9300loss:0.5520983934402466
Epoch9300loss:0.7254190444946289
Epoch9300loss:0.6094177961349487
Epoch9400loss:0.8789310455322266
Epoch9400loss:0.8134136199951172
Epoch9400loss:0.6461362242698669
Epoch9400loss:0.5936154127120972
Epoch9400loss:0.49513205885887146
Epoch9400loss:0.5330464243888855
Epoch9400loss:0.770027756690979
Epoch9400loss:0.4513534605503082
Epoch9500loss:0.5662533640861511
Epoch9500loss:0.5771268606185913
Epoch9500loss:0.5465446710586548
Epoch9500loss:0.5646964907646179
Epoch9500loss:0.4977565407752991
Epoch9500loss:0.5647937655448914
Epoch9500loss:0.76507967710495
Epoch9500loss:0.6824004054069519
Epoch9600loss:1.2897034883499146
Epoch9600loss:0.9794045090675354
Epoch9600loss:0.6185852885246277
Epoch9600loss:0.4837118983268738
Epoch9600loss:0.500331699848175
Epoch9600loss:0.6317114233970642
Epoch9600loss:0.9763137102127075
Epoch9600loss:1.2919831275939941
Epoch9700loss:0.8233212232589722
Epoch9700loss:0.8227604627609253
Epoch9700los

最后使用该数据玩游戏

In [81]:
testX = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(1,101)])
# if torch.cuda.is_available():
#     testX = testX.cuda()
with torch.no_grad():
    testY = model(testX)
# predicts = zip(range(1,101),list(testY.max(1)[1]))    

In [82]:
pred  = testY.max(1)[1].cpu().data.tolist()

In [83]:
predicts = zip(range(1,101),pred)
predictions = np.array([fizz_buzz_decode(i,x) for i,x in predicts])
print(predictions)

['1' 'fizz' 'fizz' 'fizz' 'buzz' 'fizz' '7' '8' 'buzz' 'buzz' 'fizz'
 'fizz' '13' '14' 'fizzbuzz' '16' '17' 'fizz' 'buzz' 'buzz' 'fizz' '22'
 'fizz' 'buzz' 'buzz' 'fizz' 'fizz' '28' '29' 'fizzbuzz' '31' 'fizz'
 'fizz' 'buzz' 'buzz' 'fizz' '37' 'fizz' 'buzz' 'buzz' 'fizz' 'fizz' '43'
 '44' 'fizzbuzz' '46' 'fizz' 'fizz' 'buzz' 'buzz' 'buzz' '52' '53' 'buzz'
 'buzz' 'fizz' 'fizz' '58' '59' 'fizzbuzz' '61' 'fizz' 'buzz' 'fizz'
 'buzz' 'fizz' '67' 'buzz' 'buzz' 'buzz' 'fizz' 'fizz' '73' '74'
 'fizzbuzz' 'fizz' '77' 'fizz' 'buzz' 'buzz' 'fizz' '82' 'fizz' 'buzz'
 'buzz' 'fizz' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' 'buzz' 'buzz'
 'fizz' '97' 'fizz' 'buzz' 'buzz']


In [84]:
trY_100 = np.array([fizz_buzz_encode(i) for i in range(1,101)])
print(trY_100)

[0 0 1 0 2 1 0 0 1 2 0 1 0 0 3 0 0 1 0 2 1 0 0 1 2 0 1 0 0 3 0 0 1 0 2 1 0
 0 1 2 0 1 0 0 3 0 0 1 0 2 1 0 0 1 2 0 1 0 0 3 0 0 1 0 2 1 0 0 1 2 0 1 0 0
 3 0 0 1 0 2 1 0 0 1 2 0 1 0 0 3 0 0 1 0 2 1 0 0 1 2]


In [85]:
a = (pred == trY_100)
pred = np.array(pred)
print("正确率：",pred[a].mean())


正确率： 0.9411764705882353
